# start

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from collections import Counter
import os

In [3]:
bad = pd.read_csv('/content/drive/MyDrive/trans/good_bad split/bad_features.csv')
good = pd.read_csv('/content/drive/MyDrive/trans/good_bad split/good_features.csv')

In [53]:
good.head(1)

,Unnamed: 0,uid,make_sense_count,make_sense_p_exp,tutor_frust,stu_frust,stu_frust_p_exp,total_tech_score,informal_p,senti_p,feedback_p,effective_q_p,stu_utt_p,stu_da_dist,tu_da_dist,all_da_dist
0,0,285a5de8-f157-424f-9b54-5ae097c55dfb.csv,2,0.0,3,0,-1.0,0.7,0.092683,0.056098,0.557471,0.0625,0.085366,"[0, 0, 34, 0, 1]","[145, 25, 140, 17, 48]","[145, 25, 174, 17, 49]"


In [50]:
bad.head(1)

,Unnamed: 0,uid,make_sense_count,make_sense_p_exp,tutor_frust,stu_frust,stu_frust_p_exp,total_tech_score,informal_p,senti_p,feedback_p,effective_q_p,stu_utt_p,stu_da_dist,tu_da_dist,all_da_dist
0,0,430bf65e-1b58-48a0-969d-d8a6180ae3a6.csv.csv,4,0.25,0,0,-1.0,0.55,0.238411,0.033113,0.0,0.533333,0.13245,"[0, 17, 0, 1, 2]","[1, 38, 3, 74, 15]","[1, 55, 3, 75, 17]"


# Frequency data

In [44]:
def count_da(d):
  result = []
  categories = ['Explanation', 'Extra Domain', 'Feedback', 'Other', 'Questions']
  for i in categories:
    result.append(len(d[d['DA'] == i]))
  return result

In [48]:
dir = '/content/drive/MyDrive/trans/good_bad split/good_da/'
good_names = os.listdir(dir)

stu_utt_p = []
stu_da_dist = []
tu_da_dist = []
all_da_dist = []

for j in good_names:
  d = pd.read_csv(dir + j)

  # stu_utt_p
  ct_str = d['Speaker'].value_counts()
  if 'student' in ct_str.index:
    stu_utt_p.append(ct_str['student']/len(d))
  else:
    stu_utt_p.append(0)

  # all_da_dist
  all_da_dist.append(count_da(d))

  s_da = d[d['Speaker'] == 'student']
  t_da = d[d['Speaker'] == 'tutor']

  stu_da_dist.append(count_da(s_da))
  tu_da_dist.append(count_da(t_da))

good['stu_utt_p'] = stu_utt_p
good['stu_da_dist'] = stu_da_dist
good['tu_da_dist'] = tu_da_dist
good['all_da_dist'] = all_da_dist

In [49]:
dir = '/content/drive/MyDrive/trans/good_bad split/bad_da/'
bad_names = os.listdir(dir)

stu_utt_p = []
stu_da_dist = []
tu_da_dist = []
all_da_dist = []

for j in bad_names:
  d = pd.read_csv(dir + j)

  # stu_utt_p
  ct_str = d['Speaker'].value_counts()
  if 'student' in ct_str.index:
    stu_utt_p.append(ct_str['student']/len(d))
  else:
    stu_utt_p.append(0)

  # all_da_dist
  all_da_dist.append(count_da(d))

  s_da = d[d['Speaker'] == 'student']
  t_da = d[d['Speaker'] == 'tutor']

  stu_da_dist.append(count_da(s_da))
  tu_da_dist.append(count_da(t_da))

bad['stu_utt_p'] = stu_utt_p
bad['stu_da_dist'] = stu_da_dist
bad['tu_da_dist'] = tu_da_dist
bad['all_da_dist'] = all_da_dist

# test

In [95]:
def sum_dist(d):
  position_sums = [0] * 5
  for lst in d:
      position_sums = [sum(x) for x in zip(position_sums, lst)]
  return position_sums
def prob(d):
  if sum(d) == 0:
    return [0] * 5
  else:
    return [(x / sum(d)) for x in d]

In [133]:
good_stu = good['stu_da_dist'].apply(lambda x: prob(x))
good_tu = good['tu_da_dist'].apply(lambda x: prob(x))
good_all = good['all_da_dist'].apply(lambda x: prob(x))
bad_stu = bad['stu_da_dist'].apply(lambda x: prob(x))
bad_tu = bad['tu_da_dist'].apply(lambda x: prob(x))
bad_all = bad['all_da_dist'].apply(lambda x: prob(x))


## sig

In good sessions, students partcipate more actively (reasonable, because it is student rating)

In [6]:
stats.ttest_ind(good['make_sense_p_exp'], bad['make_sense_p_exp'], alternative='greater')

TtestResult(statistic=2.7223838797679885, pvalue=0.003561153397368768, df=179.0)

In [7]:
stats.ttest_ind(good['effective_q_p'], bad['effective_q_p'], alternative='greater')

TtestResult(statistic=2.1014434542159033, pvalue=0.01850092481834777, df=179.0)

In [140]:
stats.ttest_ind([i[4] for i in good_stu], [i[4] for i in bad_stu], alternative='greater') # question da

TtestResult(statistic=2.7492022888512215, pvalue=0.0032934040063064434, df=179.0)

different da distribution across but also not consistant (too small sample size)

In [75]:
chi2_contingency([sum_dist(bad['all_da_dist']), sum_dist(good['all_da_dist'])])

Chi2ContingencyResult(statistic=46.007528535077974, pvalue=2.4539825414438446e-09, dof=4, expected_freq=array([[18404.37678262,  3986.52282403, 17398.4532593 ,  1891.90264176,
         5665.74449229],
       [29622.62321738,  6416.47717597, 28003.5467407 ,  3045.09735824,
         9119.25550771]]))

In [76]:
chi2_contingency([sum_dist(bad['tu_da_dist']), sum_dist(good['tu_da_dist'])])

Chi2ContingencyResult(statistic=41.71803933130643, pvalue=1.9084181668066117e-08, dof=4, expected_freq=array([[15463.36922953,  2631.1045311 ,  9836.4486621 ,  1964.65902648,
         4693.41855078],
       [23331.63077047,  3969.8954689 , 14841.5513379 ,  2964.34097352,
         7081.58144922]]))

In [77]:
chi2_contingency([sum_dist(bad['stu_da_dist']), sum_dist(good['stu_da_dist'])])

Chi2ContingencyResult(statistic=33.067730623528625, pvalue=1.1569409593091767e-06, dof=4, expected_freq=array([[3.20276971e+03, 1.31899160e+03, 7.18923274e+03, 2.77536370e+00,
        1.04423059e+03],
       [6.02923029e+03, 2.48300840e+03, 1.35337673e+04, 5.22463630e+00,
        1.96576941e+03]]))

## not sig

In [52]:
stats.ttest_ind(good['stu_utt_p'], bad['stu_utt_p'], alternative='greater')

TtestResult(statistic=0.318504898264214, pvalue=0.6247633506269784, df=179.0)

In [8]:
stats.ttest_ind(good['make_sense_count'], bad['make_sense_count'], alternative='greater')

TtestResult(statistic=0.6848977734331949, pvalue=0.24714716595860253, df=179.0)

In [9]:
stats.ttest_ind(good['tutor_frust'], bad['tutor_frust'], alternative='less')

TtestResult(statistic=2.4947420263893028, pvalue=0.9932446047705021, df=179.0)

In [10]:
stats.ttest_ind(good['stu_frust'], bad['stu_frust'], alternative='less')

TtestResult(statistic=0.29368234881181987, pvalue=0.6153295605746052, df=179.0)

In [14]:
stats.ttest_ind(good['stu_frust_p_exp'], bad['stu_frust_p_exp'], alternative='greater')

TtestResult(statistic=1.397524334686983, pvalue=0.08199282258100186, df=179.0)

In [12]:
stats.ttest_ind(good['total_tech_score'], bad['total_tech_score'], alternative='greater')

TtestResult(statistic=-0.00849414447769329, pvalue=0.5033839029929358, df=179.0)

In [15]:
stats.ttest_ind(good['informal_p'], bad['informal_p'], alternative='less')

TtestResult(statistic=0.6553969966640272, pvalue=0.7434731959282295, df=179.0)

In [16]:
stats.ttest_ind(good['senti_p'], bad['senti_p'], alternative='greater')

TtestResult(statistic=0.821321626108495, pvalue=0.20627781933429107, df=179.0)

In [17]:
stats.ttest_ind(good['feedback_p'], bad['feedback_p'], alternative='greater')

TtestResult(statistic=1.0285897842394296, pvalue=0.1525301840703125, df=179.0)

### tutor da dist

In [127]:
stats.ttest_ind([i[0] for i in good_tu], [i[0] for i in bad_tu])

TtestResult(statistic=0.23445737796467936, pvalue=0.8148978825990856, df=179.0)

In [128]:
stats.ttest_ind([i[1] for i in good_tu], [i[1] for i in bad_tu])

TtestResult(statistic=-0.9120641522370058, pvalue=0.3629616534166238, df=179.0)

In [129]:
stats.ttest_ind([i[2] for i in good_tu], [i[2] for i in bad_tu])

TtestResult(statistic=0.8592964347823877, pvalue=0.3913262969826192, df=179.0)

In [130]:
stats.ttest_ind([i[3] for i in good_tu], [i[3] for i in bad_tu])

TtestResult(statistic=1.8005637950287903, pvalue=0.07345520661724964, df=179.0)

In [131]:
stats.ttest_ind([i[4] for i in good_tu], [i[4] for i in bad_tu])

TtestResult(statistic=0.207489330140056, pvalue=0.8358635905460305, df=179.0)

### stu dist

In [141]:
stats.ttest_ind([i[0] for i in good_stu], [i[0] for i in bad_stu])

TtestResult(statistic=1.2714935094952713, pvalue=0.2052027672558057, df=179.0)

In [142]:
stats.ttest_ind([i[1] for i in good_stu], [i[1] for i in bad_stu])

TtestResult(statistic=0.10614324024859793, pvalue=0.9155875627585499, df=179.0)

In [143]:
stats.ttest_ind([i[2] for i in good_stu], [i[2] for i in bad_stu])

TtestResult(statistic=-1.6229930600637004, pvalue=0.10635072242841, df=179.0)

In [144]:
stats.ttest_ind([i[3] for i in good_stu], [i[3] for i in bad_stu])

TtestResult(statistic=-0.9027307372548969, pvalue=0.36788198839810515, df=179.0)

### all dist

In [150]:
stats.ttest_ind([i[0] for i in good_all], [i[0] for i in bad_all])

TtestResult(statistic=0.20967873674145499, pvalue=0.8341568766161862, df=179.0)

In [151]:
stats.ttest_ind([i[1] for i in good_all], [i[1] for i in bad_all])

TtestResult(statistic=-1.5821805466530539, pvalue=0.11537365181615822, df=179.0)

In [152]:
stats.ttest_ind([i[2] for i in good_all], [i[2] for i in bad_all])

TtestResult(statistic=0.47397534749047454, pvalue=0.6360951099782154, df=179.0)

In [153]:
stats.ttest_ind([i[3] for i in good_all], [i[3] for i in bad_all])

TtestResult(statistic=0.8778547918313911, pvalue=0.38119928001397574, df=179.0)

In [154]:
stats.ttest_ind([i[4] for i in good_all], [i[4] for i in bad_all])

TtestResult(statistic=0.44595814082595, pvalue=0.6561664353227695, df=179.0)